In [1]:
import utils
(train_x, train_y), (test_x, test_y) = utils.processed_data()

In [8]:
import keras
import keras.layers as layers
import keras.models as models
import keras.backend as K


def residual_block(x, filters, strides=1, **kwargs):
    """Constructs a residual convolutional block."""
    bn1 = layers.BatchNormalization(scale=False)(x)
    relu1 = layers.Activation("relu")(bn1)
    conv1 = layers.Conv2D(filters[0], 3, strides=strides, padding="same")(relu1)
    bn2 = layers.BatchNormalization(scale=False)(conv1)
    relu2 = layers.Activation("relu")(bn2)
    conv2 = layers.Conv2D(filters[1], 3, padding="same")(relu2)
    
    if strides != 1:
        x = layers.Conv2D(filters[1], 1, strides=strides, padding="same")(x)
        
    return layers.Add()([conv2, x])


def create_resnet():
    inputs = layers.Input((28,28))
    reshape = layers.Reshape((28,28,1))(inputs)
    
    # conv layer here?
    
    res1 = residual_block(reshape, (8, 8))
    res2 = residual_block(res1, (8, 8))
    res3 = residual_block(res2, (8, 16), strides=2)
    res4 = residual_block(res3, (16, 16))
    res5 = residual_block(res4, (16, 16))
    res6 = residual_block(res5, (16, 32), strides=2)
    res7 = residual_block(res6, (32, 32))
    
    pooling = layers.GlobalMaxPooling2D()(res4)
    outputs = layers.Dense(10, activation="softmax")(pooling)
    return models.Model(inputs=inputs, outputs=outputs)


model = create_resnet()

model.compile(
    optimizer=keras.optimizers.SGD(lr=1e-3),
    loss="categorical_crossentropy",
    metrics=[keras.metrics.categorical_accuracy]
)

history = model.fit(
        x=train_x,
        y=train_y,
        batch_size=256,
        epochs=20,
        validation_data=(test_x, test_y),
        verbose=1,
        shuffle=True
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 5s 91us/step - loss: 2.6383 - categorical_accuracy: 0.1648 - val_loss: 2.1288 - val_categorical_accuracy: 0.2206
Epoch 2/20
60000/60000 [==============================] - 3s 54us/step - loss: 1.9981 - categorical_accuracy: 0.2979 - val_loss: 1.8939 - val_categorical_accuracy: 0.3448
Epoch 3/20
60000/60000 [==============================] - 3s 54us/step - loss: 1.7969 - categorical_accuracy: 0.3893 - val_loss: 1.7108 - val_categorical_accuracy: 0.4182
Epoch 4/20
60000/60000 [==============================] - 3s 54us/step - loss: 1.6395 - categorical_accuracy: 0.4476 - val_loss: 1.5785 - val_categorical_accuracy: 0.4703
Epoch 5/20
60000/60000 [==============================] - 3s 54us/step - loss: 1.5189 - categorical_accuracy: 0.4937 - val_loss: 1.4676 - val_categorical_accuracy: 0.5070
Epoch 6/20
60000/60000 [==============================] - 3s 55us/step - loss: 1.4196 - categor